In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.common.keys import Keys
#from selenium.common.exceptions import NoSuchElementException


In [3]:
# Configuración del navegador
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_options.add_argument("--disable-blink-features=AutomationControlled")
c_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Inicializar el navegador
c_driver = webdriver.Chrome(options=c_options)

# URL de la página TeslaHunt
url = "https://teslahunt.io/?country=Euro"
c_driver.get(url)
time.sleep(3)  # Esperar a que cargue la página


In [4]:
# Esperar que los elementos carguen
wait = WebDriverWait(c_driver, 20)
time.sleep(5)  # Espera inicial para asegurar la carga

In [5]:
# Diccionario para almacenar los resultados
results = {
    "Modelo": [],
    "Precio": [],
    "Kilómetros": [],
    "Año": [],
    "Color": []
}



In [6]:
try:
    # Scroll dinámico
    last_height = c_driver.execute_script("return document.body.scrollHeight")
    for _ in range(10):
        c_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = c_driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Seleccionar todos los anuncios
    ads = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.css-1yz7e9k")))
    print(f"Número de anuncios encontrados: {len(ads)}")

    # Iterar sobre los anuncios
    results = {
        "Modelo": [],
        "Precio": [],
        "Kilómetros": [],
        "Año": [],
        "Color": []
    }

    for ad in ads:
        try:
            # Modelo del coche
            model = ad.find_element(By.CSS_SELECTOR, "p.css-1yuuaeh").text

            # Precio
            price = ad.find_element(By.CSS_SELECTOR, "p.css-iq7v1d").text.split("•")[0]

            # Kilómetros y Año
            details = ad.find_element(By.CSS_SELECTOR, "p.css-iq7v1d").text
            year = details.split("•")[1].strip() if "•" in details else "No disponible"
            km = details.split("•")[2].replace("kms", "").strip() if "•" in details else "No disponible"

            # Color (primero que coincida)
            buttons = ad.find_elements(By.CSS_SELECTOR, "button.css-npbpoy")
            color = "No disponible"
            for button in buttons:
                if "color" in button.text.lower():
                    color = button.text.replace("#", "").strip()

            # Guardar en el diccionario
            results["Modelo"].append(model)
            results["Precio"].append(price)
            results["Kilómetros"].append(km)
            results["Año"].append(year)
            results["Color"].append(color)

            print(f"Modelo: {model}, Precio: {price}, Km: {km}, Año: {year}, Color: {color}")
        except Exception as e:
            print(f"Error en anuncio: {e}")

except Exception as e:
    print(f"No se pudieron obtener los anuncios: {e}")

# Crear DataFrame resultante
df_teslahunt = pd.DataFrame(results)
print(df_teslahunt)







Número de anuncios encontrados: 222
Modelo: Tesla Model 3 Standard Range Plus, Precio: 22,700€, Km: 94,669, Año: 2020, Color: pearl white multi-coat color
Error en anuncio: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p.css-1yuuaeh"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60808FB05+28789]
	(No symbol) [0x00007FF607FF86E0]
	(No symbol) [0x00007FF607E9592A]
	(No symbol) [0x00007FF607EE930E]
	(No symbol) [0x00007FF607EE95FC]
	(No symbol) [0x00007FF607EDC6EC]
	(No symbol) [0x00007FF607F0F47F]
	(No symbol) [0x00007FF607EDC5B6]
	(No symbol) [0x00007FF607F0F650]
	(No symbol) [0x00007FF607F2F654]
	(No symbol) [0x00007FF607F0F1E3]
	(No symbol) [0x00007FF607EDA938]
	(No symbol) [0x00007FF607EDBAA1]
	GetHandleVerifier [0x00007FF6083C933D+3410093]
	GetHandleVerifier [0x

In [8]:
df_teslahunt

,Modelo,Precio,Kilómetros,Año,Color
0,Tesla Model 3 Standard Range Plus,"22,700€","94,669",2020,pearl white multi-coat color
1,Tesla Model 3 Standard Range Plus,"22,700€","94,669",2020,pearl white multi-coat color
2,Tesla Model 3 Standard Range Plus,"23,100€","83,529",2020,pearl white multi-coat color
3,Tesla Model 3 Standard Range Plus,"23,200€","59,815",2020,pearl white multi-coat color
4,Tesla Model 3 Standard Range Plus,"23,400€","71,171",2020,deep blue metallic color
...,...,...,...,...,...
216,Tesla Model 3 Long Range AWD,"29,800€","35,252",2020,red multi-coat color
217,Tesla Model 3 Long Range AWD,"29,800€","38,354",2020,pearl white multi-coat color
218,Tesla Model 3 Long Range AWD,"29,800€","41,507",2020,solid black color
219,Tesla Model 3 Standard Range Plus,"29,800€","71,431",2020,pearl white multi-coat color


In [9]:
# Contar las ocurrencias de "No disponible"
for column in df_teslahunt.columns:
    count_no_disponible = df_teslahunt[column].value_counts().get("No disponible", 0)
    print(f'Columna "{column}": {count_no_disponible} ocurrencias de "No disponible"')


Columna "Modelo": 0 ocurrencias de "No disponible"
Columna "Precio": 0 ocurrencias de "No disponible"
Columna "Kilómetros": 0 ocurrencias de "No disponible"
Columna "Año": 0 ocurrencias de "No disponible"
Columna "Color": 61 ocurrencias de "No disponible"


In [12]:
# Guardar los resultados en un archivo Excel
df_teslahunt.to_excel("teslahunt_results.xlsx", index=False)

In [13]:
# Cerrar navegador
c_driver.quit()